# Data Cleaning for FAPS Individual PUF

### Imports

In [126]:
import numpy as np
import pandas as pd

### Data

In [127]:
df = pd.read_csv('faps_individual_puf.csv', low_memory=False)

In [128]:
df.shape

(14317, 77)

In [129]:
df.head()

,HHNUM,PNUM,INITINTRVMON,INITFINALDAYS,TSSTRATA,TSPSU,HHWGT,GUEST,FAMMEMBER,SEX,...,INCRETDISINDAVG_R,INCRETDISINDAVG_FLAG,INCINVESTINDAVG_R,INCINVESTINDAVG_FLAG,INCOTHERINDAVG_R,INCOTHERINDAVG_FLAG,HOMETENURE,YEARSRESIDENCE,USBORN,USCITIZEN
0,100012,1,1,8,15,14,6056.913412,0,1,1,...,0,0,0,0,0,0,0,1,1,V
1,100012,2,1,8,15,14,6056.913412,0,1,2,...,0,0,0,0,0,0,0,1,1,V
2,100012,3,1,8,15,14,6056.913412,0,1,1,...,0,0,0,0,0,0,0,1,1,V
3,100012,4,1,8,15,14,6056.913412,0,1,2,...,V,V,V,V,V,V,V,V,V,V
4,100012,5,1,8,15,14,6056.913412,0,1,1,...,V,V,V,V,V,V,V,V,V,V


In [130]:
# Number of households
df.groupby('HHNUM').ngroups

4826

### FAMMEMBER - Individual is a member of the primary respondent's family

* 0 - Not in Primary Respondent's Family
* 1 - Part of Primary Respondent's Family

Note: Identifies individuals that are part of the PR’s family when 
calculating family poverty guideline and threshold and when 
calculating total family income.

Dropping any non-family members and guests from households

In [131]:
df['FAMMEMBER'].value_counts()

1    13202
0     1115
Name: FAMMEMBER, dtype: int64

In [132]:
df = df[df['FAMMEMBER'] == 1] # Filtering out non-family members

### Relation_R

Check that Primary Respondent is PNUM = 1

In [133]:
df['RELATION_R'].value_counts()

0    4825
3    4776
1    2021
4     442
7     442
5     401
6     295
Name: RELATION_R, dtype: int64

In [134]:
df[df['RELATION_R'] == '0']['PNUM'].value_counts()
# Number of households with primary respondent

1    4825
Name: PNUM, dtype: int64

This confirms that the primary respondent always has PNUM = 1

In [135]:
df['RELATION_R'].value_counts()

0    4825
3    4776
1    2021
4     442
7     442
5     401
6     295
Name: RELATION_R, dtype: int64

In [136]:
len(df.groupby('HHNUM')) # Number of households

4826

This result is weird. There should be as many primary respondents as there are households. However, there is one more household than primary respondents. Upon further investigation, I discovered that it is due to HHNUM 117741. This is a super unique case. Looking back at the original data, the 'Primary Respondent' is a Guest, which was previously filtered out. Next, there is a second person in the household who is not a guest, but is also not a family member. The only non-guest family member is a child/stepchild/adopted child of the primary respondent who is a guest. This seems like such a weird household that it can be eliminated from the dataset.

In [137]:
w = df[['HHNUM','PNUM']].groupby('HHNUM').min()['PNUM']
w[w != 1] # Getting household without primary respondent

HHNUM
117741    3
Name: PNUM, dtype: int64

In [138]:
# Original members of household 117741
pd.read_csv('faps_individual_puf.csv', low_memory=False).iloc[[12645,12646,12647]][['HHNUM', 'PNUM', 'GUEST', 'FAMMEMBER', 'RELATION_R']]

,HHNUM,PNUM,GUEST,FAMMEMBER,RELATION_R
12645,117741,1,1,0,0
12646,117741,2,0,0,2
12647,117741,3,0,1,3


In [139]:
# Dropping household 117741
df.drop(index = 12647, inplace = True)

### SEX

#### Method 1: Calculate the percent of household that is female

* 1 - Male
* 2 - Female
* R - Refused

There was one refusal, but they were dropped in FAMMEMBER section

In [140]:
print(df['SEX'].value_counts())
df['SEX'] = pd.to_numeric(df['SEX'])

2    7121
1    6080
Name: SEX, dtype: int64


In [141]:
df['SEX'] = df['SEX'].apply(lambda x: x - 1) # Converts sex to 0, 1 binary

# Where we create our df_hh dataframe to track demographic characteristics of households
df_hh = pd.DataFrame(df.groupby('HHNUM').mean()['SEX'])
df_hh.rename(columns = {'SEX':'PERCENT_FEMALE'}, inplace = True)
df_hh

,PERCENT_FEMALE
HHNUM,
100012,0.400000
100015,0.000000
100024,0.500000
100026,0.500000
100028,0.333333
...,...
120049,0.500000
120067,0.333333
120077,1.000000


#### Method 2: Sex of Primary Respondent
0. Male primary respondent
1. Female primary respondent

In [142]:
# Getting sex of primary respondent
df_hh['PR_SEX'] = np.asarray(df[df['PNUM'] == 1]['SEX'])
df_hh['PR_SEX'].value_counts()

1    3547
0    1278
Name: PR_SEX, dtype: int64

### AGE_R

This one is a little weird since AGE_R is the approximate midpoint of 
individual’s age group



#### Method 1: Getting the average age of each household

In [143]:
df['AGE_R'].value_counts()

47.5    3746
27.5    2929
9.5      949
13.5     908
62.5     713
85       690
6.5      470
16.5     455
68       434
5        282
3        277
4        267
2        261
1        240
18       189
0        188
19       182
40        20
D          1
Name: AGE_R, dtype: int64

In [144]:
# Dropping Don't Know Response
age_df = df[df['AGE_R'] != 'D']

In [145]:
age_df['AVG_AGE_R'] = pd.to_numeric(age_df['AGE_R'])

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\1056581010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df['AVG_AGE_R'] = pd.to_numeric(age_df['AGE_R'])


In [146]:
# Getting average age of all household members
age_col = pd.DataFrame(age_df.groupby('HHNUM').mean()['AVG_AGE_R'])
df_hh = pd.concat([df_hh, age_col], axis = 1)
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R
HHNUM,,,
100012,0.400000,0,29.100000
100015,0.000000,0,62.500000
100024,0.500000,1,32.000000
100026,0.500000,1,27.500000
100028,0.333333,1,26.583333
...,...,...,...
120049,0.500000,1,14.250000
120067,0.333333,1,23.416667
120077,1.000000,1,85.000000


#### Method 2: Getting the age of the primary respondent

In [147]:
df_hh['PR_AGE_R'] = np.array(age_df[age_df['PNUM'] == 1]['AGE_R'])

In [148]:
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R
HHNUM,,,,
100012,0.400000,0,29.100000,47.5
100015,0.000000,0,62.500000,62.5
100024,0.500000,1,32.000000,47.5
100026,0.500000,1,27.500000,27.5
100028,0.333333,1,26.583333,47.5
...,...,...,...,...
120049,0.500000,1,14.250000,27.5
120067,0.333333,1,23.416667,27.5
120077,1.000000,1,85.000000,85


### HISPANIC - Create a variable for percentage of household that is hispanic

RECODE-indicates individual Hispanic 

* 0 - No
* 1 - Yes
* R - Refused
* D - Don't know

#### Method 1: First, remove those who answered R or D. Next, calculate the percentage of HH that is hispanic.
* ISSUE: Two households are then missing

In [149]:
df['HISPANIC'].value_counts()

0    9698
1    3491
R      11
D       1
Name: HISPANIC, dtype: int64

In [150]:
df_hispanic = df[(df['HISPANIC'] != 'R') & (df['HISPANIC'] != 'D')]
df_hispanic['HISPANIC'] = pd.to_numeric(df_hispanic['HISPANIC'])

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\3986956343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hispanic['HISPANIC'] = pd.to_numeric(df_hispanic['HISPANIC'])


In [151]:
hispanic = df_hispanic.groupby('HHNUM').mean()['HISPANIC']
# 2 households contain members who all responded R or D
print(hispanic.shape, ': This should be (4825,)')

(4823,) : This should be (4825,)


#### Method 2: Calculate probability of someone being hispanic. Assign that value to those who responded R or D. Then, calulate proportion of HH that is hispanic.

In [152]:
# Probability of being hispanic in the sample
prob_hispanic = df_hispanic['HISPANIC'].mean()
prob_hispanic

0.2646902721965274

In [153]:
df_alt = df # Alternate dataframe to make this easier

# Assigning R and D responders value of prob_hispanic
df_alt['HISPANIC'] = df_alt['HISPANIC'].apply(lambda x: bool(x)*0 + prob_hispanic if (x == 'R' or x == 'D') else x)
print(df_alt['HISPANIC'].value_counts())
df_alt['HISPANIC'] = pd.to_numeric(df_alt['HISPANIC'])

0                     9698
1                     3491
0.2646902721965274      12
Name: HISPANIC, dtype: int64


In [154]:
# Proportion of household members that are hispanic
hispanic = df_alt.groupby('HHNUM').mean()['HISPANIC']

In [155]:
df_hh = pd.concat([df_hh, hispanic], axis = 1)
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC
HHNUM,,,,,
100012,0.400000,0,29.100000,47.5,0.0
100015,0.000000,0,62.500000,62.5,0.0
100024,0.500000,1,32.000000,47.5,0.0
100026,0.500000,1,27.500000,27.5,0.0
100028,0.333333,1,26.583333,47.5,0.0
...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0
120067,0.333333,1,23.416667,27.5,0.0
120077,1.000000,1,85.000000,85,0.0


### RACECAT_R - Racial Category of individual

* 1 - White
* 2 - Black
* 3 - American Indian or Alaska Native
* 4 - Asian or Native Hawaiian or Other Pacific Islander
* 5 - Other Race
* 6 - Multiple Race
* -998 - Refused

Here, four new indicators for RaceCat_R are created:

0. Refused (See below)
1. White
2. Black
3. Other

In [156]:
df_racecat = pd.DataFrame(df[['HHNUM','RACECAT_R','PNUM']]) # Work with smaller dataframe

df_racecat['RACECAT_R'].replace({'R':0}, inplace=True)
df_racecat['RACECAT_R'] = pd.DataFrame(pd.to_numeric(df_racecat['RACECAT_R']))
df_racecat['RACECAT_R'].value_counts()

1    8832
2    1983
5    1383
4     525
6     326
3     131
0      21
Name: RACECAT_R, dtype: int64

21 respondents refused to provide their race in the survey. Of those 21 respondents:
* 15 belong to households with other races
    * The other members of the household will determine the race variable of the household
* 6 are the only member of their household
    * These households will be encoded with a value of 0 and removed during the final data pre-processing

Removal of 15 refusals with more than one household member

In [157]:
df_racecat = df_racecat[(df_racecat['RACECAT_R'] != 0) | (df_racecat['PNUM'] == 1)]

Recode categories 3 through 6 as Other (3)

In [158]:
df_racecat['RACECAT_R'].replace({3:3, 4:3, 5:3, 6:3}, inplace=True)
df_racecat['RACECAT_R'].value_counts()

1    8832
3    2365
2    1983
0       6
Name: RACECAT_R, dtype: int64

#### Method 1: Code the household based on the MODE of RaceCat_R

In [159]:
df_racecat_method1 = df_racecat[['RACECAT_R', 'HHNUM']].groupby('HHNUM').agg(lambda x: x.value_counts().index[0])

In [160]:
print(df_racecat_method1.shape)
df_racecat_method1.value_counts()

(4825, 1)


RACECAT_R
1            3346
3             761
2             713
0               5
dtype: int64

In [161]:
df_hh['MODE_RACECAT_R'] = df_racecat_method1['RACECAT_R']
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC,MODE_RACECAT_R
HHNUM,,,,,,
100012,0.400000,0,29.100000,47.5,0.0,1
100015,0.000000,0,62.500000,62.5,0.0,2
100024,0.500000,1,32.000000,47.5,0.0,1
100026,0.500000,1,27.500000,27.5,0.0,1
100028,0.333333,1,26.583333,47.5,0.0,1
...,...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0,2
120067,0.333333,1,23.416667,27.5,0.0,1
120077,1.000000,1,85.000000,85,0.0,1


#### Method 2: Create proportion variables: Proportion White, Proportion Black, Proportion Other

Tracks the proportion in each household of each race category

* alt_pnum tracks actual number of people per household after filtering for family members previously

In [163]:
df_alt_pnum = df_racecat.groupby('HHNUM').count()['PNUM']
df_hh['ALT_PNUM'] = df_alt_pnum

##### Indicator Variables for Each Category

In [164]:
df_racecat_dum = pd.concat([df_racecat, pd.get_dummies(df_racecat['RACECAT_R'])], axis=1)
df_racecat_dum

,HHNUM,RACECAT_R,PNUM,0,1,2,3
0,100012,1,1,0,1,0,0
1,100012,1,2,0,1,0,0
2,100012,1,3,0,1,0,0
3,100012,1,4,0,1,0,0
4,100012,1,5,0,1,0,0
...,...,...,...,...,...,...,...
14312,120067,1,7,0,1,0,0
14313,120077,1,1,0,1,0,0
14314,120078,2,1,0,0,1,0
14315,120078,2,2,0,0,1,0


##### Proportion of White HH Members

In [165]:
df_hh['PROP_WHITE'] = df_racecat_dum.groupby('HHNUM').sum()[1] / df_alt_pnum

##### Proportion of Black HH Members

In [166]:
df_hh['PROP_BLACK'] = df_racecat_dum.groupby('HHNUM').sum()[2] / df_alt_pnum

##### Proportion of Members of Other Races

In [167]:
df_hh['PROP_OTHER'] = df_racecat_dum.groupby('HHNUM').sum()[3] / df_alt_pnum

##### Proportion of Missing Values

In [168]:
df_hh['PROP_NARACE'] = df_racecat_dum.groupby('HHNUM').sum()[0] / df_alt_pnum

In [169]:
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC,MODE_RACECAT_R,ALT_PNUM,PROP_WHITE,PROP_BLACK,PROP_OTHER,PROP_NARACE
HHNUM,,,,,,,,,,,
100012,0.400000,0,29.100000,47.5,0.0,1,5,1.0,0.0,0.0,0.0
100015,0.000000,0,62.500000,62.5,0.0,2,1,0.0,1.0,0.0,0.0
100024,0.500000,1,32.000000,47.5,0.0,1,2,1.0,0.0,0.0,0.0
100026,0.500000,1,27.500000,27.5,0.0,1,2,1.0,0.0,0.0,0.0
100028,0.333333,1,26.583333,47.5,0.0,1,6,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0,2,2,0.0,1.0,0.0,0.0
120067,0.333333,1,23.416667,27.5,0.0,1,6,1.0,0.0,0.0,0.0
120077,1.000000,1,85.000000,85,0.0,1,1,1.0,0.0,0.0,0.0


CHECK: The sum of each category per household is 1

In [170]:
(df_hh['PROP_WHITE'] + df_hh['PROP_NARACE'] + df_hh['PROP_OTHER'] + df_hh['PROP_BLACK']).value_counts()

1.0    4825
dtype: int64

### EDUCCAT -  Individual's highest level of completed education

Universe: AGE_R>=16, -998 or -997

* 1 - 10th grade or less
* 2 - 11th or 12th grade, no diploma
* 3 - H.S. diploma, GED or equivalent
* 4 - Some college or associate’s degree
* 5 - Bachelor’s degree
* 6 - Master's degree and above
* -997 - Don’t know
* -998 - Refused
* -996 - Valid skip

3 Groups Are Created Here

0. No info
1. More than high school
2. High school only
3. Less than high school

In [171]:
df_educ = pd.DataFrame(df[['HHNUM','EDUCCAT','PNUM']])
# Encode values as they appear in documentation
df_educ['EDUCCAT'].replace({'V':-996, 'D': -997, 'R': -998}, inplace=True)
df_educ['EDUCCAT'] = pd.DataFrame(pd.to_numeric(df_educ['EDUCCAT']))
df_educ['EDUCCAT'].value_counts()

-996    3842
 3      2848
 4      2560
 1      1383
 5      1143
 2       826
 6       560
-997      29
-998      10
Name: EDUCCAT, dtype: int64

Check to see if any households do not have a member in the universe

In [172]:
df_educ_max = df_educ.groupby('HHNUM').max()
df_educ_max[df_educ_max['EDUCCAT'] < 0]

,EDUCCAT,PNUM
HHNUM,,
109624,-998,1
114319,-996,4


It appears there are two households who have no members with education information.

For now, these will be encoded as 0 and likely removed during the final data pre-processing.

#### Method 1: Take the member of the household with the highest education and use that as our data point

In [173]:
print(len(df_educ_max))
df_educ_max['EDUCCAT'].value_counts()

4825


 4      1688
 3      1302
 5       803
 6       461
 1       334
 2       235
-998       1
-996       1
Name: EDUCCAT, dtype: int64

Creating new groups

In [174]:
df_educ_max['EDUCCAT'].replace({-998:0, -997:0, -996:0, 1:3, 2:3, 3:2, 4:1, 5:1, 6:1}, inplace = True)
df_educ_max['EDUCCAT'].value_counts()

1    2952
2    1302
3     569
0       2
Name: EDUCCAT, dtype: int64

In [175]:
df_hh = pd.concat([df_hh, df_educ_max['EDUCCAT']], axis = 1)
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC,MODE_RACECAT_R,ALT_PNUM,PROP_WHITE,PROP_BLACK,PROP_OTHER,PROP_NARACE,EDUCCAT
HHNUM,,,,,,,,,,,,
100012,0.400000,0,29.100000,47.5,0.0,1,5,1.0,0.0,0.0,0.0,1
100015,0.000000,0,62.500000,62.5,0.0,2,1,0.0,1.0,0.0,0.0,2
100024,0.500000,1,32.000000,47.5,0.0,1,2,1.0,0.0,0.0,0.0,1
100026,0.500000,1,27.500000,27.5,0.0,1,2,1.0,0.0,0.0,0.0,3
100028,0.333333,1,26.583333,47.5,0.0,1,6,1.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0,2,2,0.0,1.0,0.0,0.0,1
120067,0.333333,1,23.416667,27.5,0.0,1,6,1.0,0.0,0.0,0.0,2
120077,1.000000,1,85.000000,85,0.0,1,1,1.0,0.0,0.0,0.0,2


#### Method 2: Getting the education of the primary respondent

In [176]:
# Value counts for Primary Respondents
pr_educ = df_educ[df_educ['PNUM'] == 1]['EDUCCAT']
pr_educ.value_counts()

 4      1574
 3      1388
 5       697
 1       554
 6       325
 2       284
-998       3
Name: EDUCCAT, dtype: int64

In [177]:
# Recoding
pr_educ.replace({-998:0, -997:0, -996:0, 1:3, 2:3, 3:2, 4:1, 5:1, 6:1}, inplace = True)
pr_educ.value_counts()

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\406273341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_educ.replace({-998:0, -997:0, -996:0, 1:3, 2:3, 3:2, 4:1, 5:1, 6:1}, inplace = True)


1    2596
2    1388
3     838
0       3
Name: EDUCCAT, dtype: int64

In [178]:
df_hh['PR_EDUCCAT'] = np.array(pr_educ)
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC,MODE_RACECAT_R,ALT_PNUM,PROP_WHITE,PROP_BLACK,PROP_OTHER,PROP_NARACE,EDUCCAT,PR_EDUCCAT
HHNUM,,,,,,,,,,,,,
100012,0.400000,0,29.100000,47.5,0.0,1,5,1.0,0.0,0.0,0.0,1,2
100015,0.000000,0,62.500000,62.5,0.0,2,1,0.0,1.0,0.0,0.0,2,2
100024,0.500000,1,32.000000,47.5,0.0,1,2,1.0,0.0,0.0,0.0,1,1
100026,0.500000,1,27.500000,27.5,0.0,1,2,1.0,0.0,0.0,0.0,3,3
100028,0.333333,1,26.583333,47.5,0.0,1,6,1.0,0.0,0.0,0.0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0,2,2,0.0,1.0,0.0,0.0,1,1
120067,0.333333,1,23.416667,27.5,0.0,1,6,1.0,0.0,0.0,0.0,2,2
120077,1.000000,1,85.000000,85,0.0,1,1,1.0,0.0,0.0,0.0,2,2


### USCITIZEN - Individual is a U.S. citizen - This is used to get non-citizen immigrant indicator

Universe: Final Interview, question I3a=“Not in U.S”, “refused”, “don’t know,” or was missing

Website link: https://ers.usda.gov/data-products/foodaps-national-household-food-acquisition-and-purchase-survey/documentation/

Interview form link: https://www.ers.usda.gov/media/8619/finalinterview.pdf

Question I3a = Where were you born?

Hence, anyone who answers no to US citizen is a non-citizen immigrant

* 0 - No
* 1 - Yes
* . - Missing
* -997 - Don't know
* -998 - Refused
* -999 - Valid Skip

In [179]:
df['USCITIZEN'].value_counts(dropna=False)

V      11336
0       1024
1        799
NaN       32
R          9
D          1
Name: USCITIZEN, dtype: int64

In [180]:
df_citizen = df[['HHNUM', 'USCITIZEN', 'PNUM']]

Create 3 categories for non-citizen immigrant indicator:
* 0 - Not a non-citizen immigrant (Are a citizen whether immigrant or not)
* 1 - Yes a non-citizen immigrant (Born outside the US and not a citizen)
* 2 - Maybe a non-citizen immigrant (Made up of those who answered Missing, Don't Know, or Refused)
    * NOTE: This is originally encoded as -1 and then changed later

In [181]:
df_citizen['USCITIZEN'].fillna(-1, inplace=True)
df_citizen.replace({'0':1, '1':0, 'D':-1, 'R':-1, 'V':0, '2':-1}, inplace=True)
df_citizen['USCITIZEN'].value_counts()

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\4066867226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_citizen['USCITIZEN'].fillna(-1, inplace=True)
C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\4066867226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_citizen.replace({'0':1, '1':0, 'D':-1, 'R':-1, 'V':0, '2':-1}, inplace=True)


 0    12135
 1     1024
-1       42
Name: USCITIZEN, dtype: int64

#### Method 1: Households with any member classified as non-citizen immigrant will be encoded as 1

In [182]:
grouped_max = df_citizen.groupby('HHNUM').max()['USCITIZEN']
grouped_max.value_counts()

 0    4199
 1     624
-1       2
Name: USCITIZEN, dtype: int64

In [183]:
# Recoding 'Maybe' to 2
grouped_max.replace({-1:2}, inplace=True)
grouped_max.value_counts()

0    4199
1     624
2       2
Name: USCITIZEN, dtype: int64

In [184]:
df_hh['NCIMMIGRANT_ANY'] = np.array(grouped_max)
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC,MODE_RACECAT_R,ALT_PNUM,PROP_WHITE,PROP_BLACK,PROP_OTHER,PROP_NARACE,EDUCCAT,PR_EDUCCAT,NCIMMIGRANT_ANY
HHNUM,,,,,,,,,,,,,,
100012,0.400000,0,29.100000,47.5,0.0,1,5,1.0,0.0,0.0,0.0,1,2,0
100015,0.000000,0,62.500000,62.5,0.0,2,1,0.0,1.0,0.0,0.0,2,2,0
100024,0.500000,1,32.000000,47.5,0.0,1,2,1.0,0.0,0.0,0.0,1,1,0
100026,0.500000,1,27.500000,27.5,0.0,1,2,1.0,0.0,0.0,0.0,3,3,0
100028,0.333333,1,26.583333,47.5,0.0,1,6,1.0,0.0,0.0,0.0,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0,2,2,0.0,1.0,0.0,0.0,1,1,0
120067,0.333333,1,23.416667,27.5,0.0,1,6,1.0,0.0,0.0,0.0,2,2,0
120077,1.000000,1,85.000000,85,0.0,1,1,1.0,0.0,0.0,0.0,2,2,0


#### Method 2: Households with every member classified as non-citizen immigrant will be encoded as 1

Households where status is maybe coding changed to 2

In [185]:
df_citizen['USCITIZEN'].replace({-1:2}, inplace=True)
df_citizen['USCITIZEN'].value_counts()

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\588165577.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_citizen['USCITIZEN'].replace({-1:2}, inplace=True)


0    12135
1     1024
2       42
Name: USCITIZEN, dtype: int64

It appears that there are two households where all respondents are in the maybe category. These respondents will be coded with a 2 and likely removed during final data pre-processing.

In [186]:
grouped_min = df_citizen.groupby('HHNUM').min()['USCITIZEN']
grouped_min.value_counts()

0    4671
1     152
2       2
Name: USCITIZEN, dtype: int64

In [187]:
grouped_min[grouped_min == 2]

HHNUM
112223    2
118585    2
Name: USCITIZEN, dtype: int64

In [188]:
df_hh['NCIMMIGRANT_EVERY'] = np.array(grouped_min)
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC,MODE_RACECAT_R,ALT_PNUM,PROP_WHITE,PROP_BLACK,PROP_OTHER,PROP_NARACE,EDUCCAT,PR_EDUCCAT,NCIMMIGRANT_ANY,NCIMMIGRANT_EVERY
HHNUM,,,,,,,,,,,,,,,
100012,0.400000,0,29.100000,47.5,0.0,1,5,1.0,0.0,0.0,0.0,1,2,0,0
100015,0.000000,0,62.500000,62.5,0.0,2,1,0.0,1.0,0.0,0.0,2,2,0,0
100024,0.500000,1,32.000000,47.5,0.0,1,2,1.0,0.0,0.0,0.0,1,1,0,0
100026,0.500000,1,27.500000,27.5,0.0,1,2,1.0,0.0,0.0,0.0,3,3,0,0
100028,0.333333,1,26.583333,47.5,0.0,1,6,1.0,0.0,0.0,0.0,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0,2,2,0.0,1.0,0.0,0.0,1,1,0,0
120067,0.333333,1,23.416667,27.5,0.0,1,6,1.0,0.0,0.0,0.0,2,2,0,0
120077,1.000000,1,85.000000,85,0.0,1,1,1.0,0.0,0.0,0.0,2,2,0,0


### Number of Children - This will be computed myself

Look at AGE_R. For any AGE_R < 18, these people will be counted as children. For any AGE_R >= 18, these will be counted as adults.

First, create a new indicator column for children. Then, create an indicator column for adults. Then, groupby household and sum.

There is one person that responded 'Don't Know' to age. Since they are listed as the family member 1 out of 4, I will assume they responded to the form and are an adult.

In [189]:
df[df['AGE_R'] == 'D']

,HHNUM,PNUM,INITINTRVMON,INITFINALDAYS,TSSTRATA,TSPSU,HHWGT,GUEST,FAMMEMBER,SEX,...,INCRETDISINDAVG_R,INCRETDISINDAVG_FLAG,INCINVESTINDAVG_R,INCINVESTINDAVG_FLAG,INCOTHERINDAVG_R,INCOTHERINDAVG_FLAG,HOMETENURE,YEARSRESIDENCE,USBORN,USCITIZEN
2104,102824,4,8,12,25,40,6144.935212,0,1,1,...,0,1,0,1,1,1,0,1,0,0


In [190]:
child_adult_df = df[['HHNUM','AGE_R','PNUM']]

In [191]:
child_adult_df['AGE_R'].replace({'D':50}, inplace=True)

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\1732830935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_adult_df['AGE_R'].replace({'D':50}, inplace=True)


In [192]:
child_adult_df['AGE_R'] = pd.to_numeric(child_adult_df['AGE_R'])

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\178677989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_adult_df['AGE_R'] = pd.to_numeric(child_adult_df['AGE_R'])


In [193]:
# Encoding dummy for child
child_adult_df['NCHILDREN'] = child_adult_df['AGE_R'].apply(lambda x: 1 if x < 18.0 else 0)
child_adult_df['NCHILDREN'].value_counts()

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\1452194922.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_adult_df['NCHILDREN'] = child_adult_df['AGE_R'].apply(lambda x: 1 if x < 18.0 else 0)


0    8904
1    4297
Name: NCHILDREN, dtype: int64

In [194]:
# Encoding dummy for adult
child_adult_df['NADULTS'] = child_adult_df['AGE_R'].apply(lambda x: 0 if x < 18.0 else 1)
child_adult_df['NADULTS'].value_counts()

C:\Users\samuel.thomas\AppData\Local\Temp\ipykernel_11996\532289879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_adult_df['NADULTS'] = child_adult_df['AGE_R'].apply(lambda x: 0 if x < 18.0 else 1)


1    8904
0    4297
Name: NADULTS, dtype: int64

In [195]:
# Calculating number of adults and children in household
child_adult_cols = child_adult_df.groupby('HHNUM').sum()[['NCHILDREN', 'NADULTS']]

In [196]:
df_hh = pd.concat([df_hh, child_adult_cols], axis = 1)
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC,MODE_RACECAT_R,ALT_PNUM,PROP_WHITE,PROP_BLACK,PROP_OTHER,PROP_NARACE,EDUCCAT,PR_EDUCCAT,NCIMMIGRANT_ANY,NCIMMIGRANT_EVERY,NCHILDREN,NADULTS
HHNUM,,,,,,,,,,,,,,,,,
100012,0.400000,0,29.100000,47.5,0.0,1,5,1.0,0.0,0.0,0.0,1,2,0,0,2,3
100015,0.000000,0,62.500000,62.5,0.0,2,1,0.0,1.0,0.0,0.0,2,2,0,0,0,1
100024,0.500000,1,32.000000,47.5,0.0,1,2,1.0,0.0,0.0,0.0,1,1,0,0,1,1
100026,0.500000,1,27.500000,27.5,0.0,1,2,1.0,0.0,0.0,0.0,3,3,0,0,0,2
100028,0.333333,1,26.583333,47.5,0.0,1,6,1.0,0.0,0.0,0.0,3,3,0,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0,2,2,0.0,1.0,0.0,0.0,1,1,0,0,1,1
120067,0.333333,1,23.416667,27.5,0.0,1,6,1.0,0.0,0.0,0.0,2,2,0,0,2,4
120077,1.000000,1,85.000000,85,0.0,1,1,1.0,0.0,0.0,0.0,2,2,0,0,0,1


### Disabled Person in Household

Disabled persons according to the study are defined on page 24 under section 2.3.8.3.2. They are defined as follows:
1. Individuals under age 60 (AGE_R < 62.5)and either
    1. An SSI Recipient (INCREDISIND > 0 and INC4TYPESP 'SSI') - Cannot determine whether someone is disabled or retired based on PUF

    or

    2. Not working or not in school because disabled (REASONNOWORK = 5 or SCHLEVEL = 10 [Says 12 in documentation but it is not correct]) and receiving a disability-based benefit (INCRETDISIND > 0 (== 1 in PUF) and INC4TYPESP 'Black Lung, Workers Comp, or SSDI')

NOTE: INC4TYPESP is a suppressed variable

Here, we find all people who meet every condition. Then, classify their household as containing a disabled member.

In [197]:
df_disabled = pd.DataFrame(df[['HHNUM','AGE_R','REASONNOWORK','SCHLEVEL_R','INCRETDISINDAVG_R']])
df_disabled.head()

,HHNUM,AGE_R,REASONNOWORK,SCHLEVEL_R,INCRETDISINDAVG_R
0,100012,47.5,V,V,0
1,100012,47.5,V,V,0
2,100012,27.5,V,V,0
3,100012,13.5,V,4,V
4,100012,9.5,V,2,V


REASONNOWORK == 5

In [198]:
df_disabled['REASONNOWORK'] = df_disabled['REASONNOWORK'].replace({'V':-996, 'R': -998})
df_disabled['REASONNOWORK'] = pd.DataFrame(pd.to_numeric(df_disabled['REASONNOWORK']))
df_disabled['REASONNOWORK'] = df_disabled['REASONNOWORK'].apply(lambda x: x == 5) * 1
df_disabled['REASONNOWORK'].value_counts()

0    12515
1      686
Name: REASONNOWORK, dtype: int64

SCHLEVEL_R == 10

In [199]:
df_disabled['SCHLEVEL_R'] = df_disabled['SCHLEVEL_R'].replace({'V':-996})
df_disabled['SCHLEVEL_R'] = pd.DataFrame(pd.to_numeric(df_disabled['SCHLEVEL_R']))
df_disabled['SCHLEVEL_R'] = df_disabled['SCHLEVEL_R'].apply(lambda x: x == 10) * 1
df_disabled['SCHLEVEL_R'].value_counts()

0    13137
1       64
Name: SCHLEVEL_R, dtype: int64

INCRETDISINDAVG_R == 1

In [200]:
df_disabled['INCRETDISINDAVG_R'] = df_disabled['INCRETDISINDAVG_R'].replace({'V':-996})
df_disabled['INCRETDISINDAVG_R'] = pd.DataFrame(pd.to_numeric(df_disabled['INCRETDISINDAVG_R']))
df_disabled['INCRETDISINDAVG_R'] = df_disabled['INCRETDISINDAVG_R'].apply(lambda x: x == 1) * 1
df_disabled['INCRETDISINDAVG_R'].value_counts()

0    10691
1     2510
Name: INCRETDISINDAVG_R, dtype: int64

AGE_R < 60

In [201]:
df_disabled['AGE_R'] = df_disabled['AGE_R'].replace({'D':-996})
df_disabled['AGE_R'] = pd.DataFrame(pd.to_numeric(df_disabled['AGE_R']))
df_disabled['AGE_R'] = df_disabled['AGE_R'].apply(lambda x: x < 60) * 1
df_disabled['AGE_R'].value_counts()

1    11364
0     1837
Name: AGE_R, dtype: int64

#### Getting All Individuals with Conditions Met. Grouping by household. Combining with df_hh

In [202]:
df_disabled['DISABLED'] = ((df_disabled['AGE_R'] == 1) & ((df_disabled['REASONNOWORK'] == 1) | (df_disabled['SCHLEVEL_R'] == 1)) & (df_disabled['INCRETDISINDAVG_R'] == 1)) * 1
df_disabled['DISABLED'].value_counts() # 430 disabled individuals in the dataset

0    12771
1      430
Name: DISABLED, dtype: int64

In [203]:
df_disabled_col = df_disabled.groupby('HHNUM').max()['DISABLED']
df_disabled_col.value_counts() # 398 households with at least one disabled member

0    4427
1     398
Name: DISABLED, dtype: int64

In [204]:
df_hh = pd.concat([df_hh, df_disabled_col], axis = 1)
df_hh

,PERCENT_FEMALE,PR_SEX,AVG_AGE_R,PR_AGE_R,HISPANIC,MODE_RACECAT_R,ALT_PNUM,PROP_WHITE,PROP_BLACK,PROP_OTHER,PROP_NARACE,EDUCCAT,PR_EDUCCAT,NCIMMIGRANT_ANY,NCIMMIGRANT_EVERY,NCHILDREN,NADULTS,DISABLED
HHNUM,,,,,,,,,,,,,,,,,,
100012,0.400000,0,29.100000,47.5,0.0,1,5,1.0,0.0,0.0,0.0,1,2,0,0,2,3,0
100015,0.000000,0,62.500000,62.5,0.0,2,1,0.0,1.0,0.0,0.0,2,2,0,0,0,1,0
100024,0.500000,1,32.000000,47.5,0.0,1,2,1.0,0.0,0.0,0.0,1,1,0,0,1,1,0
100026,0.500000,1,27.500000,27.5,0.0,1,2,1.0,0.0,0.0,0.0,3,3,0,0,0,2,0
100028,0.333333,1,26.583333,47.5,0.0,1,6,1.0,0.0,0.0,0.0,3,3,0,0,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120049,0.500000,1,14.250000,27.5,0.0,2,2,0.0,1.0,0.0,0.0,1,1,0,0,1,1,0
120067,0.333333,1,23.416667,27.5,0.0,1,6,1.0,0.0,0.0,0.0,2,2,0,0,2,4,0
120077,1.000000,1,85.000000,85,0.0,1,1,1.0,0.0,0.0,0.0,2,2,0,0,0,1,0


### Write to CSV

In [205]:
df_hh.to_csv('faps_individual_clean.csv')